In [45]:
import multiprocessing
import math

from sympy import *

In [46]:
def is_sorted(lst):
    for i in range(len(lst) - 1):
        if lst[i] >= lst[i+1]:
            return False
    return True

In [47]:
assert is_sorted([1,2,3])
assert not is_sorted([3,2,1])

In [48]:
# Constant to be set before running
CORES = 8 # Number of CPU cores to use
n = 74 # Desired number of primes in total
k = 3 # Number of last primes to vary
DESIRED_BITS = 512

# Computed constants
DESIRED_SIZE = 2**DESIRED_BITS
f = n - k # Number of fixed primes

fixed_primes = [prime(i) for i in range(1, f + 1)]
assert len(fixed_primes) == f
largest_fixed = max(fixed_primes)
initial_product = 4 * 3 * prod(fixed_primes)

# Computing the search space
search_start = nextprime(largest_fixed)

search_end = nextprime(700) # More than enough for a prime of our size
# A fully exhaustive search would use the formula below:
#search_end = prevprime(DESIRED_SIZE / (initial_product * prod([prime(f + 1) for i in range(k - 1)])))

candidate_primes = list(primerange(search_start, search_end + 1))
assert is_sorted(candidate_primes)
max_tuple_prod = DESIRED_SIZE / initial_product

# Start-up info
print('Searching for a prime p of at most', DESIRED_BITS, 'bits')
print('p + 1 should have', n, 'distinct prime factors')
print('Fixed the first', f, 'smallest prime factors')
print('Searching for best candidates for the last', k, 'prime factors')
print('Largest fixed prime is', largest_fixed)
print('Searching for primes between', search_start, 'and', search_end)
print('There are', len(candidate_primes), 'primes in this range')
print('Search space has', binomial(len(candidate_primes), k), 'elements')

Searching for a prime p of at most 512 bits
p + 1 should have 74 distinct prime factors
Fixed the first 71 smallest prime factors
Searching for best candidates for the last 3 prime factors
Largest fixed prime is 353
Searching for primes between 359 and 701
There are 55 primes in this range
Search space has 26235 elements


In [49]:
N(log(max_tuple_prod * initial_product - 1, 2))

512.000000000000

In [50]:
def possible_next_values(tup):
    max_prime = max(tup)
    search_start = candidate_primes.index(max_prime)
    stop_at = floor(max_tuple_prod / prod(tup))
    adds = []
    for i in range(search_start + 1, len(candidate_primes)):
        candidate = candidate_primes[i]
        if candidate > stop_at:
            break
        adds.append(candidate)
    return adds


def possible_tuples_starting_with(starting):
    tuples = [(starting,)]
    for i in range(1, k):
        new_tuples = []
        for t in tuples:
            assert len(t) == i
            next_vals = possible_next_values(t)
            for n in next_vals:
                new_tuples.append(t + (n,))
        tuples = new_tuples
    return tuples

In [51]:
# Since search_end is small we could just take all combinations
# However this code works in the general case with a larger value of search_end

with multiprocessing.Pool(CORES) as pool:
    possible_tuples_lists = pool.map(possible_tuples_starting_with, candidate_primes)

possible_tuples = []
for tuple_list in possible_tuples_lists:
    possible_tuples.extend(tuple_list)

for tup in possible_tuples:
    assert is_sorted(tup)

In [52]:
def scored_tuple(tup):
    return (tup, sum(math.sqrt(x) for x in tup))

In [53]:
with multiprocessing.Pool(CORES) as pool:
    scored_tuples = pool.map(scored_tuple, possible_tuples)

scored_tuples.sort(key=lambda x : x[1])

In [62]:
if len(scored_tuples) > 0:
    for tup, score in scored_tuples:
        p = 3 * 4 * initial_product * prod(tup) - 1
        if isprime(p):
            break

    print('Success:', isprime(p) and p % 8 == 7)
    print(p)
    print(N(log(p, 2)), 'bits')
else:
    print('No primes found')

Success: True
733940726498676569288807405759528571941506543729480876913520789114849260368218281657211345304565266472168410171749235447680320260228240973463600588854879
507.808733977719 bits


In [60]:
print('p + 1 prime factors:')
print(list(factorint(p + 1).keys()))

p + 1 prime factors:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 419]
